In [37]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator #Data Augmentation
from tensorflow.keras.callbacks import EarlyStopping


In [50]:
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=0.2,shear_range=0.2,zoom_range=0.2,
                                width_shift_range=0.2,
                                height_shift_range=0.2,validation_split=0.2)#validation split splits the training dataset into ratio of 80 and 20% in which 80% is used for training and 20% is used for validating the model on the go


In [71]:
train_data=train_datagen.flow_from_directory(r'D:\pythonnn\dataset\train_dataset',
                                             target_size=(80,80),
                                             batch_size=8,
                                             class_mode='categorical',subset='training')
validation_data=train_datagen.flow_from_directory(r'D:\pythonnn\dataset\train_dataset',
                                             target_size=(80,80),
                                             batch_size=8,
                                             class_mode='categorical',subset='validation')

Found 34560 images belonging to 2 classes.
Found 8640 images belonging to 2 classes.


In [72]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_data=test_datagen.flow_from_directory(r'D:\pythonnn\dataset\test_dataset',
                                             target_size=(80,80),
                                             batch_size=8,
                                             class_mode='categorical')

Found 4800 images belonging to 2 classes.


In [73]:
bmodel= InceptionV3(include_top='false',
                    weights='imagenet',
                    input_tensor=Input(shape=[80,80,3]))#basically exclude the top layer because we only need the knowledge og the inceptionv3 model and not its outputlayer 
#weights tells us which knowledge to borrow from here we are borrowiing knowledge from imagenet dataset
hmodel=bmodel.output
hmodel=Flatten()(hmodel)
hmodel=Dense(64,activation='relu')(hmodel)#In a neural network, a Dense layer (also known as a fully connected layer) is a fundamental layer where each neuron is connected to every neuron in the previous layer. The Dense layer's primary purpose is to perform learned transformations on the input data
hmodel=Dropout(0.5)(hmodel)#to prevent overfeeding
hmodel=Dense(2,activation='softmax')(hmodel)#final output we need only 2 neurons
model=Model(inputs=bmodel.input,outputs=hmodel)
#since we used knowledge of inceptionv3 so we only need to train the last layers which we added according to our requirements
for layer in bmodel.layers:
    layer.trainable=False
#this basically is done because the in between layers we only use them and no need to train them hence trainable set to false for the layersn in the base model



In [74]:
model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_11 (InputLayer)   │ (None, 80, 80, 3)         │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_940 (Conv2D)           │ (None, 39, 39, 32)        │             864 │ input_layer_11[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_940       │ (None, 39, 39, 32)        │              96 │ conv2d_940[0][0]           │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_940 (Activation)   │ (None, 39, 39, 32)        │               0 │ batch_normalization_940[0… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_941 (Conv2D)           │ (None, 37, 37, 32)        │           9,216 │ activation_940[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_941       │ (None, 37, 37, 32)        │              96 │ conv2d_941[0][0]           │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_941 (Activation)   │ (None, 37, 37, 32)        │               0 │ batch_normalization_941[0… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_942 (Conv2D)           │ (None, 37, 37, 64)        │          18,432 │ activation_941[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_942       │ (None, 37, 37, 64)        │             192 │ conv2d_942[0][0]           │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_942 (Activation)   │ (None, 37, 37, 64)        │               0 │ batch_normalization_942[0… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_40              │ (None, 18, 18, 64)        │               0 │ activation_942[0][0]       │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_943 (Conv2D)           │ (None, 18, 18, 80)        │           5,120 │ max_pooling2d_40[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_943       │ (None, 18, 18, 80)        │             240 │ conv2d_943[0][0]           │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_943 (Activation)   │ (None, 18, 18, 80)        │               0 │ batch_normalization_943[0… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_944 (Conv2D)           │ (None, 16, 16, 192)       │         138,24

 Total params: 23,915,978 (91.23 MB)

 Trainable params: 64,194 (250.76 KB)

 Non-trainable params: 23,851,784 (90.99 MB)

In [54]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau


In [77]:
checkpoint=ModelCheckpoint(r'D:\pythonnn\models\model.keras' , monitor='val_loss' 
                           , save_best_only=True,verbose=3)
earlystop=EarlyStopping(monitor='val_loss',patience=7,verbose=3,restore_best_weights=True)
#patience =7 means if accuracy is not increasing then stop
#when training for a very long time but the accuracy is not increasing or the validation loss is not decreasing then stop training the model
learning_rate=ReduceLROnPlateau(monitor='val_loss',patience=3,verbose=3)
callbacks=[checkpoint,earlystop,learning_rate]

In [78]:
batchsize=8
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
if validation_data is None:
    raise ValueError("validation_data is None")
if train_data.samples < batchsize or validation_data.samples < batchsize:
    raise ValueError("Batch size is larger than dataset size")
model.fit(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                   epochs=5)
if steps_per_epoch == 0 or validation_steps == 0:
    raise ValueError("steps_per_epoch or validation_steps are zero, check your batchsize and dataset.")

Epoch 1/5
4320/4320 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.9105 - loss: 0.2196
Epoch 1: val_loss improved from inf to 0.23827, saving model to D:\pythonnn\models\model.keras
4320/4320 ━━━━━━━━━━━━━━━━━━━━ 418s 90ms/step - accuracy: 0.9105 - loss: 0.2196 - val_accuracy: 0.9021 - val_loss: 0.2383 - learning_rate: 0.0010
Epoch 2/5


AttributeError: 'NoneType' object has no attribute 'items'

In [59]:
acc_tr, loss_tr = model.evaluate(train_data)
print(acc_tr)
print(loss_tr)

4320/4320 ━━━━━━━━━━━━━━━━━━━━ 470s 109ms/step - accuracy: 0.9208 - loss: 0.1893
0.19085578620433807
0.9198495149612427


In [61]:
acc_vr, loss_vr = model.evaluate(validation_data)
print(acc_vr)
print(loss_vr)

1080/1080 ━━━━━━━━━━━━━━━━━━━━ 74s 68ms/step - accuracy: 0.9152 - loss: 0.2161
0.22299502789974213
0.9108796119689941


In [62]:
acc_test, loss_test = model.evaluate(test_data)
print(acc_tr)
print(loss_tr)

600/600 ━━━━━━━━━━━━━━━━━━━━ 60s 99ms/step - accuracy: 0.7759 - loss: 0.4199
0.19085578620433807
0.9198495149612427
